### import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir) # 해당 dir에 학습해둔 모델 있는지 확인
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [17]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    test_data = np.expand_dims(test_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

### Performance function

In [52]:
def loss_func(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                   y_true=labels,
                                                                   from_logits=True)) # softmax를 거치지 않을때
    return loss

def accuracy_func(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_func(model, images, labels)
    return tape.gradient(loss, model.variables)

### Model function

In [53]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, # 출력값의 크기
                                 use_bias=True, # 편향 사용 유무
                                 kernel_initializer=weight_init) # 가중치 초기화 함수

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

### Create model(class version)

In [54]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())
            
        self.model.add(dense(label_dim, weight_init))
        
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Create model(function version)

In [55]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256, weight_init))
        model.add(sigmoid())
        
    model.add(dense(label_dim, weight_init))
    
    return model

### Define data & hyper-parameter

In [56]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10
train_flag = True

# Graph input using Dataset API
# prefetch : data 로드 시간을 줄이기 위해 미리 batch size 만큼 적재시킴
# drop_remainder : 마지막 남은 data를 drop 여부
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                                                   shuffle(buffer_size=100000).\
                                                   prefetch(buffer_size=batch_size).\
                                                   batch(batch_size, drop_remainder=True) 


test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                                                  shuffle(buffer_size=100000).\
                                                  prefetch(buffer_size=len(test_x)).\
                                                  batch(len(test_x))

### Define model & optimizer & writer

In [57]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [58]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
        
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_func(network, train_input, train_label)
                train_accuracy = accuracy_func(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_func(network, test_input, test_label)
                    
                tf.summary.scalar(name='train_lass', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))   
# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_func(network, test_input, test_label)
        
    print("test Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.0063, train_loss: 2.28831768, train_accuracy: 0.1172, test_Accuracy: 0.1328
Epoch: [ 0] [    1/  468] time: 1.2678, train_loss: 2.28576183, train_accuracy: 0.1250, test_Accuracy: 0.1135
Epoch: [ 0] [    2/  468] time: 1.4864, train_loss: 2.25382042, train_accuracy: 0.1875, test_Accuracy: 0.1135
Epoch: [ 0] [    3/  468] time: 1.6622, train_loss: 2.25974751, train_accuracy: 0.1328, test_Accuracy: 0.1135
Epoch: [ 0] [    4/  468] time: 1.8280, train_loss: 2.32725525, train_accuracy: 0.0625, test_Accuracy: 0.1135
Epoch: [ 0] [    5/  468] time: 1.9950, train_loss: 2.24187779, train_accuracy: 0.2344, test_Accuracy: 0.1934
Epoch: [ 0] [    6/  468] time: 2.1580, train_loss: 2.29317188, train_accuracy: 0.0781, test_Accuracy: 0.0986
Epoch: [ 0] [    7/  468] time: 2.3278, train_loss: 2.22624588, train_accuracy: 0.1172, test_Accuracy: 0.0983
Epoch: [ 0] [    8/  468] time: 2.4986,

Epoch: [ 0] [   75/  468] time: 14.6507, train_loss: 0.94270223, train_accuracy: 0.7891, test_Accuracy: 0.7972
Epoch: [ 0] [   76/  468] time: 14.8245, train_loss: 1.01296067, train_accuracy: 0.7578, test_Accuracy: 0.7973
Epoch: [ 0] [   77/  468] time: 14.9982, train_loss: 0.93284786, train_accuracy: 0.7734, test_Accuracy: 0.7910
Epoch: [ 0] [   78/  468] time: 15.2461, train_loss: 1.03115344, train_accuracy: 0.7266, test_Accuracy: 0.7900
Epoch: [ 0] [   79/  468] time: 15.4794, train_loss: 0.88907421, train_accuracy: 0.8047, test_Accuracy: 0.7897
Epoch: [ 0] [   80/  468] time: 15.6707, train_loss: 0.90056080, train_accuracy: 0.7500, test_Accuracy: 0.7964
Epoch: [ 0] [   81/  468] time: 15.8444, train_loss: 0.89487875, train_accuracy: 0.7812, test_Accuracy: 0.8026
Epoch: [ 0] [   82/  468] time: 16.0210, train_loss: 0.89601183, train_accuracy: 0.8203, test_Accuracy: 0.8080
Epoch: [ 0] [   83/  468] time: 16.1889, train_loss: 0.83055222, train_accuracy: 0.8281, test_Accuracy: 0.8144
E

Epoch: [ 0] [  150/  468] time: 28.7725, train_loss: 0.55311882, train_accuracy: 0.8984, test_Accuracy: 0.8812
Epoch: [ 0] [  151/  468] time: 28.9355, train_loss: 0.47950593, train_accuracy: 0.8672, test_Accuracy: 0.8803
Epoch: [ 0] [  152/  468] time: 29.1112, train_loss: 0.47291833, train_accuracy: 0.8984, test_Accuracy: 0.8810
Epoch: [ 0] [  153/  468] time: 29.2976, train_loss: 0.41419399, train_accuracy: 0.8984, test_Accuracy: 0.8822
Epoch: [ 0] [  154/  468] time: 29.5006, train_loss: 0.42468131, train_accuracy: 0.8750, test_Accuracy: 0.8828
Epoch: [ 0] [  155/  468] time: 29.6997, train_loss: 0.44010213, train_accuracy: 0.8672, test_Accuracy: 0.8830
Epoch: [ 0] [  156/  468] time: 29.8978, train_loss: 0.41168666, train_accuracy: 0.9219, test_Accuracy: 0.8825
Epoch: [ 0] [  157/  468] time: 30.0881, train_loss: 0.45768064, train_accuracy: 0.8438, test_Accuracy: 0.8828
Epoch: [ 0] [  158/  468] time: 30.2951, train_loss: 0.35527420, train_accuracy: 0.9609, test_Accuracy: 0.8826
E

Epoch: [ 0] [  224/  468] time: 42.9665, train_loss: 0.44174260, train_accuracy: 0.8906, test_Accuracy: 0.9039
Epoch: [ 0] [  225/  468] time: 43.1431, train_loss: 0.39483500, train_accuracy: 0.8516, test_Accuracy: 0.9041
Epoch: [ 0] [  226/  468] time: 43.3188, train_loss: 0.40669310, train_accuracy: 0.8906, test_Accuracy: 0.9046
Epoch: [ 0] [  227/  468] time: 43.4935, train_loss: 0.33449215, train_accuracy: 0.8906, test_Accuracy: 0.9044
Epoch: [ 0] [  228/  468] time: 43.6604, train_loss: 0.38352859, train_accuracy: 0.9219, test_Accuracy: 0.9037
Epoch: [ 0] [  229/  468] time: 43.8351, train_loss: 0.38284194, train_accuracy: 0.9062, test_Accuracy: 0.9033
Epoch: [ 0] [  230/  468] time: 44.0089, train_loss: 0.34285229, train_accuracy: 0.8984, test_Accuracy: 0.9019
Epoch: [ 0] [  231/  468] time: 44.1738, train_loss: 0.34824216, train_accuracy: 0.8984, test_Accuracy: 0.9017
Epoch: [ 0] [  232/  468] time: 44.3495, train_loss: 0.42267942, train_accuracy: 0.8828, test_Accuracy: 0.9015
E

Epoch: [ 0] [  298/  468] time: 56.6959, train_loss: 0.29509535, train_accuracy: 0.9375, test_Accuracy: 0.9114
Epoch: [ 0] [  299/  468] time: 56.8686, train_loss: 0.31362236, train_accuracy: 0.8984, test_Accuracy: 0.9099
Epoch: [ 0] [  300/  468] time: 57.0453, train_loss: 0.33755216, train_accuracy: 0.9141, test_Accuracy: 0.9100
Epoch: [ 0] [  301/  468] time: 57.2190, train_loss: 0.34737402, train_accuracy: 0.9062, test_Accuracy: 0.9107
Epoch: [ 0] [  302/  468] time: 57.3908, train_loss: 0.32190120, train_accuracy: 0.8984, test_Accuracy: 0.9114
Epoch: [ 0] [  303/  468] time: 57.5636, train_loss: 0.36064920, train_accuracy: 0.8594, test_Accuracy: 0.9112
Epoch: [ 0] [  304/  468] time: 57.7392, train_loss: 0.33926532, train_accuracy: 0.8828, test_Accuracy: 0.9114
Epoch: [ 0] [  305/  468] time: 57.9149, train_loss: 0.27729571, train_accuracy: 0.9297, test_Accuracy: 0.9116
Epoch: [ 0] [  306/  468] time: 58.0925, train_loss: 0.24783391, train_accuracy: 0.9375, test_Accuracy: 0.9125
E

Epoch: [ 0] [  445/  468] time: 83.6832, train_loss: 0.42503288, train_accuracy: 0.8750, test_Accuracy: 0.9223
Epoch: [ 0] [  446/  468] time: 83.9067, train_loss: 0.17398150, train_accuracy: 0.9688, test_Accuracy: 0.9228
Epoch: [ 0] [  447/  468] time: 84.1176, train_loss: 0.27743071, train_accuracy: 0.9297, test_Accuracy: 0.9236
Epoch: [ 0] [  448/  468] time: 84.2981, train_loss: 0.25790113, train_accuracy: 0.8984, test_Accuracy: 0.9243
Epoch: [ 0] [  449/  468] time: 84.4748, train_loss: 0.28144589, train_accuracy: 0.9141, test_Accuracy: 0.9230
Epoch: [ 0] [  450/  468] time: 84.6993, train_loss: 0.40868831, train_accuracy: 0.8750, test_Accuracy: 0.9217
Epoch: [ 0] [  451/  468] time: 84.8730, train_loss: 0.27439982, train_accuracy: 0.9219, test_Accuracy: 0.9208
Epoch: [ 0] [  452/  468] time: 85.0438, train_loss: 0.38518560, train_accuracy: 0.8672, test_Accuracy: 0.9220
Epoch: [ 0] [  453/  468] time: 85.2234, train_loss: 0.19772346, train_accuracy: 0.9531, test_Accuracy: 0.9219
E

### Test accuracy : 92.38%